In [ ]:
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import numpy as np
from tensorflow.keras import optimizers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# dataset https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news
# https://www.kaggle.com/datasets/hoshi7/news-sentiment-dataset

en_data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Sentiment_dataset.csv", encoding='ISO-8859-1')

In [ ]:
en_data.head()

,news_title,reddit_title,sentiment,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,1.0,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",1.0,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,1.0,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,1.0,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn dÃ¸de i trafikken i 2019,No children died in traffic accidents in Norwa...,1.0,I 1970 dÃ¸de det 560 mennesker i den norske tr...,https://www.nrk.no/trondelag/ingen-barn-dode-i...


In [ ]:
en_data["text"][3]

'Newark police: No officer fired a single shot in 2020, thanks to de-escalation program\n\nNewark police and city officials say a de-escalation training program is working, especially in a year faced with challenges.\n\nPublic Safety Director Anthony Ambrose says 2020 was the roughest year in his 34-year career in law enforcement.\n\nSix of their 1,100 officers lost their lives to COVID-19 with dozens more officers sick after being exposed on the job.\n\nThey also faced major challenges during the summer\'s anti-police brutality protests.\n\nThrough it all, Ambrose says not one officer in the city fired his or her weapon while on duty in 2020.\n\n"It was the unknown. It was the unknown that you didnâ\x80\x99t know with this disease that you were coming here every day, and these police officers and firefighters going out there, and we didn\'t know," says Ambrose.\n\nFor Newark police, it was a year of COVID-19 fears, obstacles and losses.\n\nâ\x80\x9cWe lost six police officers and goin

In [ ]:
en_data["sentiment"].value_counts()

1.0    748
0.0    100
Name: sentiment, dtype: int64

In [ ]:
label_dict = {1:'up', 0:'down'}
en_data["sentiment"] = en_data["sentiment"].map(label_dict)

In [ ]:
en_data = en_data[en_data["sentiment"] != "neutral"]

In [ ]:
en_data.head()

,news_title,reddit_title,sentiment,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,up,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",up,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,up,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,up,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn dÃ¸de i trafikken i 2019,No children died in traffic accidents in Norwa...,up,I 1970 dÃ¸de det 560 mennesker i den norske tr...,https://www.nrk.no/trondelag/ingen-barn-dode-i...


In [ ]:
en_data_list = en_data["text"].values

In [ ]:
len(en_data_list)

848

In [ ]:
from googletrans import Translator
translator = Translator()

def translate(text):
  translated_text = translator.translate(text, src='en', dest='id').text
  print(translated_text)
  return translated_text

In [ ]:
id_data = [translate(en_text) for en_text in en_data_list]

Streaming output truncated to the last 5000 lines.

â€œMalam ini dijelaskan bahwa Hak tidak memiliki solusi untuk keamanan, tetapi kami memilikinya,â€ kata Livni dalam sebuah pernyataan. â€œKami meletakkannya di atas meja empat tahun lalu, dan sekarang perlu diimplementasikan.â€

Pemimpin Partai Buruh Avi Gabbay juga melontarkan kritik keras terhadap Netanyahu, dengan mengatakan bahwa kebijakan pemerintahnya mengenai Hamas telah mengecewakan penduduk Israel.

"Selama empat tahun, Netanyahu telah meninggalkan penduduk pinggiran dan mengabaikan masalah di Gaza," katanya dalam sebuah pernyataan.

â€œPenduduk Israel selatan seharusnya tidak perlu mendengar tentang gencatan senjata dari Al Jazeera, mereka pantas mendapatkan kepemimpinan sejati yang siap menatap mata mereka dan mengatakan yang sebenarnya,â€ kata Gabbay.
Al-Jazeera TV melaporkan bahwa kesepakatan telah dicapai antara Israel dan Hamas mengenai gencatan senjata di Jalur Gaza. Kesepakatan itu kabarnya diperoleh melalui mediasi M

In [ ]:
en_data = en_data[["sentiment", "text"]]
en_data["id_text"] = id_data
en_data = en_data[["sentiment", "id_text"]]
en_data.columns = ["label", "content"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
en_data

,label,content
0,up,Investor miliarder dan bintang Shark Tank Mark...
1,up,Pengacara Jarrett Adams baru-baru ini membantu...
2,up,Hutan Hujan Amazon terkenal di seluruh dunia s...
3,up,Polisi Newark: Tidak ada petugas yang melepask...
4,up,I 1970 dÃ¸de det 560 mennesker i den norske tr...
...,...,...
843,down,Rincian menakutkan telah muncul tentang serang...
844,down,wawancara\n\nJohannesburg - 360.000 remaja dip...
845,down,Sebuah serangan kekerasan di kota pelabuhan Is...
846,down,"Seorang anak laki-laki berusia 17 tahun, terla..."


In [ ]:
#en_data.to_csv("/content/gdrive/MyDrive/Flask NLP/financial_news_mt_2.csv", index = None)

NotADirectoryError: ignored